import pandas as pd
import matplotlib.pyplot as plt
# ^^^ pyforest auto-imports - don't write above this line


# Main Question: Do American teams have more American Players than Canadian Teams and vice versa for 2018-2019? 

## Imports

In [1]:
import warnings
from functions import *
import pickle
sns.set(style="whitegrid")


Bad key "text.kerning_factor" on line 4 in
/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## Last Year Available

In [3]:
last_year_query = f"""
    SELECT DISTINCT(season)
    FROM game
"""
last_year_query_result = run_query(last_year_query)

In [7]:
last_year = last_year_query_result[-1][0]

## Canadian Teams List

### All teams plus their ID

In [29]:
# not all of the franchises are in team_info
all_teams_query = f"""
    SELECT COUNT(*)
    FROM team_info
"""
all_teams_query_result = run_query(all_teams_query)

In [30]:
all_teams_query_result

[(17,)]

In [33]:
all_teams_query_result

[(17,)]

In [31]:
# from game_team_stats table
all_teams_query2 = f"""
    SELECT COUNT(DISTINCT(team_id))
    FROM game_teams_stats
    INNER JOIN game
        ON game.game_id = game_teams_stats.game_id
    WHERE game.season = {last_year}
""" # adding the last year clause removed 3 teams
all_teams_query2_result = run_query(all_teams_query2)

In [32]:
all_teams_query2_result

[(31,)]

### Canadian Team Ids 

In [35]:
# getting the actual team_id for each team
## also found here: https://www.kaggle.com/martinellis/nhl-game-data?select=team_info.csv
all_team_ids_query = f"""
    SELECT DISTINCT(team_id)
    FROM game_teams_stats
    INNER JOIN game
        ON game.game_id = game_teams_stats.game_id
    WHERE game.season = {last_year}
"""
all_team_ids_query_result = run_query(all_team_ids_query)

In [48]:
# flatten and sort
## https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
team_ids_flattened = [x for i in all_team_ids_query_result for x in i]
team_ids_sorted = sorted(team_ids_flattened)

In [49]:
canadian_team_dict = {'Vancouver': 26, 'Ottawa': 9,
                     'Montreal': 8, "Toronto": 10, "Calgary": 20,
                     "Winnipeg": 52, "Edmonton": 22}
canadian_team_ids = list(canadian_team_dict.values())

In [50]:
canadian_team_ids

[26, 9, 8, 10, 20, 52, 22]

### Values for "American" and "Canadian"

In [51]:
player_nat_query = f"""
    SELECT DISTINCT(nationality)
    FROM player_info
"""
player_nat_query_result = run_query(player_nat_query)

In [52]:
player_nat_query_result

[('nat',),
 ('RUS',),
 ('CAN',),
 ('SWE',),
 ('LTU',),
 ('USA',),
 ('CZE',),
 ('SVN',),
 ('FIN',),
 ('LVA',),
 ('DEU',),
 ('SVK',),
 ('DNK',),
 ('CHE',),
 ('BLR',),
 ('AUT',),
 ('FRA',),
 ('NLD',),
 ('NA',),
 ('UKR',),
 ('NGA',),
 ('HRV',),
 ('NOR',)]

### Test for a single team
only has players. or maybe not

In [53]:
last_ottawa_game_query = f"""
    SELECT game_id
    FROM game_teams_stats
    WHERE team_id = 9
    ORDER BY game_id DESC
    LIMIT 1
""" # when they were the home team
last_ottawa_game_query_result = run_query(last_ottawa_game_query)

In [55]:
last_ottawa_game_query_result

[(2018021260,)]

In [67]:
ottawa_nat_query = f"""
    SELECT DISTINCT(ps.player_id), ps.nationality
    FROM player_info ps
    INNER JOIN game_shifts gs
        ON ps.player_id = gs.player_id
    WHERE game_id = 2018021260
"""
ottawa_nat_query_result = run_query(ottawa_nat_query)

In [64]:
len(ottawa_nat_query_result)

19

In [70]:
all_nats = set(x[1] for x in ottawa_nat_query_result)

In [72]:
type(all_nats)

set

In [78]:
# getting the number of each nationality and their percentage 
ottawa_nat_dict = {x:0 for x in all_nats}
for x in ottawa_nat_query_result:
    ottawa_nat_dict[x[1]] += 1

In [82]:
num_players_on_team = len(ottawa_nat_query_result)
ottawa_nat_dict_pct = {key: round(value/num_players_on_team, 2) for key, value in ottawa_nat_dict.items()}

In [83]:
ottawa_nat_dict_pct

{'SWE': 0.11, 'USA': 0.32, 'FIN': 0.05, 'CAN': 0.47, 'CHE': 0.05}

## Combining all players into a dataframe

In [84]:
# getting all teams, their last games, and their rosters for those games 


### All teams with their last game

In [118]:
last_game_roster_query = f"""
    SELECT team_id, MAX(gts.game_id)
    FROM game_teams_stats gts
    GROUP BY team_id
        HAVING MAX(gts.game_id) > 2018000000
"""
last_game_roster_query_result = run_query(last_game_roster_query)

In [120]:
len(last_game_roster_query_result)

31

### Getting and saving their rosters

In [175]:
players_in_last_game_roster_query = f"""
    SELECT team_id, game_id, pi.player_id, 
        pi.firstName, pi.lastName, pi.nationality, pi.birthCity, pi.primaryPosition
    FROM game_skater_stats gss
    INNER JOIN player_info pi
        ON gss.player_id = pi.player_id
    WHERE (team_id, game_id) IN
    (SELECT team_id, MAX(gts.game_id)
    FROM game_teams_stats gts
    GROUP BY team_id
        HAVING MAX(gts.game_id) > 2018000000
    )
"""
players_in_last_game_roster_query_result = run_query(players_in_last_game_roster_query)

In [137]:
len(players_in_last_game_roster_query_result)

849

In [144]:
# including players who didn't play in last game
length_of_player_info_table_query = f"""
    SELECT COUNT(DISTINCT(player_id))
    FROM player_info
"""
length_of_player_info_table_query_result = run_query(length_of_player_info_table_query)
length_of_player_info_table_query_result

[(1205,)]

In [133]:
# approx number of players in last game
num_of_teams = 31
players_per_team = 21
num_of_teams * players_per_team

651

#### Inputting players into dataframe

In [186]:
player_df = pd.DataFrame(data = players_in_last_game_roster_query_result,
                        columns = ['Team_id', "Game_id", "Player_id", "First_name",
                                  "Last_name", "Country_code", "Birth_city", "Primary_position"])

In [187]:
# only players, no goalie info
player_df.Primary_position.unique()

array(['C', 'D', 'RW', 'LW'], dtype=object)

In [188]:
# removing duplicates 
player_df.drop_duplicates(subset=['Player_id'], inplace=True)
player_df.reset_index(inplace=True, drop=True)
player_df.shape

(283, 8)

##### Adding Canadian Team Feature

In [189]:
# adding canada feature
canadian_team_values = list(canadian_team_dict.values())

In [190]:
canadian_team_values

[26, 9, 8, 10, 20, 52, 22]

In [191]:
in_canada = player_df['Team_id'].isin(canadian_team_values)

In [192]:
# maybe we're missing some teams. I know that was a thing for the player_info table (only 17 teams)
in_canada.sum()

62

In [193]:
# so maybe only partial results 
canadian_teams_in_df = set(x for x in player_df['Team_id'] if x in canadian_team_values)
canadian_teams_in_df

{8, 9, 10, 20, 22, 26, 52}

In [194]:
player_df['Canadian_team'] = in_canada

In [195]:
player_df.head()

,Team_id,Game_id,Player_id,First_name,Last_name,Country_code,Birth_city,Primary_position,Canadian_team
0,24,2018021256,8470612,Ryan,Getzlaf,CAN,Regina,C,False
1,24,2018021256,8476854,Hampus,Lindholm,SWE,Helsingborg,D,False
2,24,2018021256,8478466,Daniel,Sprong,NLD,Amsterdam,RW,False
3,24,2018021256,8474683,Derek,Grant,CAN,Abbotsford,C,False
4,24,2018021256,8479368,Max,Jones,USA,Rochester,LW,False


In [197]:
player_df['Team_id'].nunique()

31

#### Grouping By Team

In [200]:
# groupby sum for each country_code
player_df_groupby = player_df.groupby('Team_id').count()

In [204]:
# a lot of teams are missing a lot of players
## I probably need to redownload the data b/c it's missing A LOOOT of data
# player_df_groupby